# Parks
This dataset hosts geographical data regarding parks in Pittsburgh.

## Metric
(closer to more parks -> better)
Ignore the line below, I was practicing LaTeX this won't be in the final product.
$$\sum^{\infty}_{i=0}{x^2} $$

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib as plt
import sys
%matplotlib inline
!conda install --yes --prefix {sys.prefix} shapefile
import shapefile as shp

class Graph:
    # I will make a graph structure out of this ASAP. 
    pass
        

parks = gpd.read_file('Parks.shp')

"""
What I need to do is get the park data to be merged with the neighborhood data in some way or another.
The CSV for parks is not very neatly aligned with neighborhood. I.e., it has no "neighborhood" column at all,
which makes this kinda frustrating. Potentially see these links:
https://stackoverflow.com/questions/57153101/how-lo-overlay-shape-files-in-geopandas
https://gis.stackexchange.com/questions/299379/superposing-two-plots-with-geopandas-with-non-fill-colours-in-one-of-them
https://geopandas.org/en/stable/docs/reference/api/geopandas.overlay.html
"""

"""
Abstract idea is to build a graph data structure out of the parks in Pittsburgh and essentially run a BFS
to determine how many parks are X distance from a neighborhood. 
(check here for more info: https://towardsdatascience.com/mapping-geograph-data-in-python-610a963d2d7f)

Both the parks and neighborhoods CSVs do contain latitude and longitude points. 

What I will probably do is try to make a graph out of the shapefile. Nodes will be neighborhoods and edges will be distance to parks.
"""

neighborhoods = gpd.read_file('../neighborhood/Neighborhoods_.shp')